In [149]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import math

In [150]:
data = pd.read_csv("titanic/train.csv")

In [151]:
#data = pd.read_csv("titanic/test.csv")

In [152]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Cleaning data

In [153]:
#Map sex to 1 and 0
data['Sex'] = data['Sex'].map({'male':0, 'female':1})

In [154]:
#Normalize age, sibsp, parch, fare
data['Age'] = (data['Age'] - data['Age'].mean()) / data['Age'].std()
data['SibSp'] = (data['SibSp'] - data['SibSp'].mean()) / data['SibSp'].std()
data['Parch'] = (data['Parch'] - data['Parch'].mean()) / data['Parch'].std()
data['Fare'] = (data['Fare'] - data['Fare'].mean()) / data['Fare'].std()

In [155]:
#Covert Pclass to multiple columns
data['Pclass1'] = (data['Pclass'] == 1).astype('int64')
data['Pclass2'] = (data['Pclass'] == 2).astype('int64')
data['Pclass3'] = (data['Pclass'] == 3).astype('int64')

In [156]:
def getTicketPrefix(ticket):
    t = ticket.split()
    if (len(t) == 2):
        return t[0]
    else:
        return ''

def getTicketNumber(ticket):
    t = ticket.split()
    if (len(t) == 2):
        return t[1]
    else:
        return t[0]

In [157]:
#Convert tickets to two columns: prefix and number
data['TicketPrefix'] = data['Ticket'].apply(getTicketPrefix)
data['TicketNumber'] = data['Ticket'].apply(getTicketNumber)

In [158]:
#Convert cabin number to whether or not passenger has a cabin
data['HasCabin'] = data['Cabin'].apply(pd.isna).astype('int64')

In [159]:
#One-hot encode Embarking station and ticket prefix
embark_dummies = pd.get_dummies(data['Embarked'], prefix='Embarked')
ticket_prefix_dummies = pd.get_dummies(data['TicketPrefix'], prefix='TicketPre')

In [160]:
cols = ['Survived','Sex','Age', 'SibSp', 'Parch', 'Fare', 'HasCabin']
#cols = ['Sex','Age', 'SibSp', 'Parch', 'Fare', 'HasCabin']

In [161]:
dataset = data[cols]
dataset = dataset.join(embark_dummies)
dataset = dataset.join(ticket_prefix_dummies)

In [162]:
dataset = dataset.dropna()

In [163]:
X_data = dataset.drop('Survived', axis=1)

In [164]:
y_labels = dataset['Survived']

In [165]:
X_data.tail()

,Sex,Age,SibSp,Parch,Fare,HasCabin,Embarked_C,Embarked_Q,Embarked_S,TicketPre_,...,TicketPre_SO/C,TicketPre_SOTON/O.Q.,TicketPre_SOTON/O2,TicketPre_SOTON/OQ,TicketPre_STON/O2.,TicketPre_SW/PP,TicketPre_W./C.,TicketPre_W.E.P.,TicketPre_W/C,TicketPre_WE/P
885,1,0.640270,-0.474279,5.729626,-0.061964,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
886,0,-0.185807,-0.474279,-0.473408,-0.386454,1,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
887,1,-0.736524,-0.474279,-0.473408,-0.044356,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
889,0,-0.254646,-0.474279,-0.473408,-0.044356,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
890,0,0.158392,-0.474279,-0.473408,-0.492101,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0


In [166]:
y_labels.tail()

885    0
886    0
887    1
889    1
890    0
Name: Survived, dtype: int64

# Split to training and testing data

In [167]:
from sklearn.model_selection import train_test_split

In [168]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_labels, test_size=0.1)

# Create and train model

In [169]:
from tensorflow import keras
from tensorflow.keras import layers

In [170]:
model = keras.Sequential([
    layers.Dense(51, activation = 'relu', input_shape=[len(X_train.keys())]),
    layers.Dense(2)
])

model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             optimizer='adam',
             metrics=['accuracy'])

In [171]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 51)                2652      
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 104       
Total params: 2,756
Trainable params: 2,756
Non-trainable params: 0
_________________________________________________________________


In [172]:
batch = X_train[:10]
result = model.predict(batch)
result

array([[-0.06631295, -0.42800036],
       [-0.02865164, -0.49028152],
       [ 0.16181491, -0.27635473],
       [-0.13936508, -0.14206873],
       [ 0.18650152, -0.04967672],
       [ 0.02092548, -0.1705947 ],
       [ 0.13345373, -0.6964746 ],
       [ 0.06582842, -0.14935115],
       [ 0.14401104,  0.18622625],
       [ 0.1852    , -0.07011009]], dtype=float32)

In [173]:
model.fit(X_train, y_train, epochs=1000, verbose=2)

Train on 642 samples
Epoch 1/1000
642/642 - 0s - loss: 0.6483 - accuracy: 0.6262
Epoch 2/1000
642/642 - 0s - loss: 0.5969 - accuracy: 0.7072
Epoch 3/1000
642/642 - 0s - loss: 0.5609 - accuracy: 0.7196
Epoch 4/1000
642/642 - 0s - loss: 0.5337 - accuracy: 0.7523
Epoch 5/1000
642/642 - 0s - loss: 0.5094 - accuracy: 0.7679
Epoch 6/1000
642/642 - 0s - loss: 0.4901 - accuracy: 0.7850
Epoch 7/1000
642/642 - 0s - loss: 0.4761 - accuracy: 0.7944
Epoch 8/1000
642/642 - 0s - loss: 0.4624 - accuracy: 0.7944
Epoch 9/1000
642/642 - 0s - loss: 0.4532 - accuracy: 0.8006
Epoch 10/1000
642/642 - 0s - loss: 0.4463 - accuracy: 0.8069
Epoch 11/1000
642/642 - 0s - loss: 0.4411 - accuracy: 0.8022
Epoch 12/1000
642/642 - 0s - loss: 0.4338 - accuracy: 0.8037
Epoch 13/1000
642/642 - 0s - loss: 0.4304 - accuracy: 0.8084
Epoch 14/1000
642/642 - 0s - loss: 0.4275 - accuracy: 0.8100
Epoch 15/1000
642/642 - 0s - loss: 0.4249 - accuracy: 0.8115
Epoch 16/1000
642/642 - 0s - loss: 0.4231 - accuracy: 0.8100
Epoch 17/100

Epoch 132/1000
642/642 - 0s - loss: 0.3375 - accuracy: 0.8567
Epoch 133/1000
642/642 - 0s - loss: 0.3397 - accuracy: 0.8536
Epoch 134/1000
642/642 - 0s - loss: 0.3383 - accuracy: 0.8505
Epoch 135/1000
642/642 - 0s - loss: 0.3364 - accuracy: 0.8474
Epoch 136/1000
642/642 - 0s - loss: 0.3442 - accuracy: 0.8489
Epoch 137/1000
642/642 - 0s - loss: 0.3419 - accuracy: 0.8489
Epoch 138/1000
642/642 - 0s - loss: 0.3463 - accuracy: 0.8474
Epoch 139/1000
642/642 - 0s - loss: 0.3419 - accuracy: 0.8489
Epoch 140/1000
642/642 - 0s - loss: 0.3370 - accuracy: 0.8505
Epoch 141/1000
642/642 - 0s - loss: 0.3351 - accuracy: 0.8505
Epoch 142/1000
642/642 - 0s - loss: 0.3349 - accuracy: 0.8489
Epoch 143/1000
642/642 - 0s - loss: 0.3344 - accuracy: 0.8551
Epoch 144/1000
642/642 - 0s - loss: 0.3335 - accuracy: 0.8505
Epoch 145/1000
642/642 - 0s - loss: 0.3331 - accuracy: 0.8442
Epoch 146/1000
642/642 - 0s - loss: 0.3339 - accuracy: 0.8505
Epoch 147/1000
642/642 - 0s - loss: 0.3335 - accuracy: 0.8489
Epoch 14

642/642 - 0s - loss: 0.3134 - accuracy: 0.8738
Epoch 265/1000
642/642 - 0s - loss: 0.3253 - accuracy: 0.8551
Epoch 266/1000
642/642 - 0s - loss: 0.3152 - accuracy: 0.8723
Epoch 267/1000
642/642 - 0s - loss: 0.3096 - accuracy: 0.8707
Epoch 268/1000
642/642 - 0s - loss: 0.3088 - accuracy: 0.8692
Epoch 269/1000
642/642 - 0s - loss: 0.3076 - accuracy: 0.8676
Epoch 270/1000
642/642 - 0s - loss: 0.3078 - accuracy: 0.8676
Epoch 271/1000
642/642 - 0s - loss: 0.3081 - accuracy: 0.8723
Epoch 272/1000
642/642 - 0s - loss: 0.3077 - accuracy: 0.8785
Epoch 273/1000
642/642 - 0s - loss: 0.3082 - accuracy: 0.8692
Epoch 274/1000
642/642 - 0s - loss: 0.3060 - accuracy: 0.8738
Epoch 275/1000
642/642 - 0s - loss: 0.3071 - accuracy: 0.8707
Epoch 276/1000
642/642 - 0s - loss: 0.3062 - accuracy: 0.8660
Epoch 277/1000
642/642 - 0s - loss: 0.3067 - accuracy: 0.8629
Epoch 278/1000
642/642 - 0s - loss: 0.3080 - accuracy: 0.8707
Epoch 279/1000
642/642 - 0s - loss: 0.3120 - accuracy: 0.8769
Epoch 280/1000
642/642 

Epoch 397/1000
642/642 - 0s - loss: 0.2946 - accuracy: 0.8801
Epoch 398/1000
642/642 - 0s - loss: 0.2919 - accuracy: 0.8738
Epoch 399/1000
642/642 - 0s - loss: 0.2910 - accuracy: 0.8723
Epoch 400/1000
642/642 - 0s - loss: 0.2910 - accuracy: 0.8738
Epoch 401/1000
642/642 - 0s - loss: 0.2904 - accuracy: 0.8723
Epoch 402/1000
642/642 - 0s - loss: 0.2925 - accuracy: 0.8723
Epoch 403/1000
642/642 - 0s - loss: 0.2930 - accuracy: 0.8801
Epoch 404/1000
642/642 - 0s - loss: 0.2894 - accuracy: 0.8785
Epoch 405/1000
642/642 - 0s - loss: 0.2897 - accuracy: 0.8754
Epoch 406/1000
642/642 - 0s - loss: 0.2905 - accuracy: 0.8754
Epoch 407/1000
642/642 - 0s - loss: 0.2963 - accuracy: 0.8754
Epoch 408/1000
642/642 - 0s - loss: 0.2922 - accuracy: 0.8723
Epoch 409/1000
642/642 - 0s - loss: 0.2936 - accuracy: 0.8738
Epoch 410/1000
642/642 - 0s - loss: 0.2949 - accuracy: 0.8785
Epoch 411/1000
642/642 - 0s - loss: 0.2925 - accuracy: 0.8707
Epoch 412/1000
642/642 - 0s - loss: 0.2940 - accuracy: 0.8738
Epoch 41

642/642 - 0s - loss: 0.2775 - accuracy: 0.8847
Epoch 530/1000
642/642 - 0s - loss: 0.2773 - accuracy: 0.8832
Epoch 531/1000
642/642 - 0s - loss: 0.2766 - accuracy: 0.8785
Epoch 532/1000
642/642 - 0s - loss: 0.2775 - accuracy: 0.8816
Epoch 533/1000
642/642 - 0s - loss: 0.2768 - accuracy: 0.8801
Epoch 534/1000
642/642 - 0s - loss: 0.2778 - accuracy: 0.8801
Epoch 535/1000
642/642 - 0s - loss: 0.2771 - accuracy: 0.8816
Epoch 536/1000
642/642 - 0s - loss: 0.2798 - accuracy: 0.8785
Epoch 537/1000
642/642 - 0s - loss: 0.2794 - accuracy: 0.8832
Epoch 538/1000
642/642 - 0s - loss: 0.2813 - accuracy: 0.8832
Epoch 539/1000
642/642 - 0s - loss: 0.2791 - accuracy: 0.8832
Epoch 540/1000
642/642 - 0s - loss: 0.2880 - accuracy: 0.8754
Epoch 541/1000
642/642 - 0s - loss: 0.2839 - accuracy: 0.8801
Epoch 542/1000
642/642 - 0s - loss: 0.2786 - accuracy: 0.8832
Epoch 543/1000
642/642 - 0s - loss: 0.2818 - accuracy: 0.8816
Epoch 544/1000
642/642 - 0s - loss: 0.2792 - accuracy: 0.8816
Epoch 545/1000
642/642 

Epoch 662/1000
642/642 - 0s - loss: 0.2697 - accuracy: 0.8879
Epoch 663/1000
642/642 - 0s - loss: 0.2690 - accuracy: 0.8863
Epoch 664/1000
642/642 - 0s - loss: 0.2719 - accuracy: 0.8941
Epoch 665/1000
642/642 - 0s - loss: 0.2661 - accuracy: 0.8879
Epoch 666/1000
642/642 - 0s - loss: 0.2660 - accuracy: 0.8910
Epoch 667/1000
642/642 - 0s - loss: 0.2740 - accuracy: 0.8847
Epoch 668/1000
642/642 - 0s - loss: 0.2713 - accuracy: 0.8879
Epoch 669/1000
642/642 - 0s - loss: 0.2672 - accuracy: 0.8879
Epoch 670/1000
642/642 - 0s - loss: 0.2654 - accuracy: 0.8879
Epoch 671/1000
642/642 - 0s - loss: 0.2654 - accuracy: 0.8894
Epoch 672/1000
642/642 - 0s - loss: 0.2683 - accuracy: 0.8863
Epoch 673/1000
642/642 - 0s - loss: 0.2662 - accuracy: 0.8925
Epoch 674/1000
642/642 - 0s - loss: 0.2672 - accuracy: 0.8832
Epoch 675/1000
642/642 - 0s - loss: 0.2662 - accuracy: 0.8832
Epoch 676/1000
642/642 - 0s - loss: 0.2642 - accuracy: 0.8894
Epoch 677/1000
642/642 - 0s - loss: 0.2672 - accuracy: 0.8910
Epoch 67

642/642 - 0s - loss: 0.2589 - accuracy: 0.8925
Epoch 795/1000
642/642 - 0s - loss: 0.2593 - accuracy: 0.8988
Epoch 796/1000
642/642 - 0s - loss: 0.2702 - accuracy: 0.8832
Epoch 797/1000
642/642 - 0s - loss: 0.2578 - accuracy: 0.8925
Epoch 798/1000
642/642 - 0s - loss: 0.2586 - accuracy: 0.8956
Epoch 799/1000
642/642 - 0s - loss: 0.2597 - accuracy: 0.8910
Epoch 800/1000
642/642 - 0s - loss: 0.2587 - accuracy: 0.8894
Epoch 801/1000
642/642 - 0s - loss: 0.2594 - accuracy: 0.8894
Epoch 802/1000
642/642 - 0s - loss: 0.2668 - accuracy: 0.8972
Epoch 803/1000
642/642 - 0s - loss: 0.2580 - accuracy: 0.8972
Epoch 804/1000
642/642 - 0s - loss: 0.2563 - accuracy: 0.8879
Epoch 805/1000
642/642 - 0s - loss: 0.2572 - accuracy: 0.8863
Epoch 806/1000
642/642 - 0s - loss: 0.2586 - accuracy: 0.8879
Epoch 807/1000
642/642 - 0s - loss: 0.2585 - accuracy: 0.8910
Epoch 808/1000
642/642 - 0s - loss: 0.2577 - accuracy: 0.8956
Epoch 809/1000
642/642 - 0s - loss: 0.2574 - accuracy: 0.8894
Epoch 810/1000
642/642 

Epoch 927/1000
642/642 - 0s - loss: 0.2493 - accuracy: 0.8941
Epoch 928/1000
642/642 - 0s - loss: 0.2493 - accuracy: 0.8925
Epoch 929/1000
642/642 - 0s - loss: 0.2488 - accuracy: 0.8941
Epoch 930/1000
642/642 - 0s - loss: 0.2587 - accuracy: 0.8941
Epoch 931/1000
642/642 - 0s - loss: 0.2675 - accuracy: 0.8832
Epoch 932/1000
642/642 - 0s - loss: 0.2532 - accuracy: 0.8956
Epoch 933/1000
642/642 - 0s - loss: 0.2533 - accuracy: 0.8988
Epoch 934/1000
642/642 - 0s - loss: 0.2532 - accuracy: 0.8941
Epoch 935/1000
642/642 - 0s - loss: 0.2496 - accuracy: 0.8925
Epoch 936/1000
642/642 - 0s - loss: 0.2481 - accuracy: 0.8925
Epoch 937/1000
642/642 - 0s - loss: 0.2482 - accuracy: 0.8956
Epoch 938/1000
642/642 - 0s - loss: 0.2475 - accuracy: 0.8910
Epoch 939/1000
642/642 - 0s - loss: 0.2480 - accuracy: 0.8925
Epoch 940/1000
642/642 - 0s - loss: 0.2480 - accuracy: 0.8925
Epoch 941/1000
642/642 - 0s - loss: 0.2602 - accuracy: 0.8863
Epoch 942/1000
642/642 - 0s - loss: 0.2514 - accuracy: 0.8925
Epoch 94

In [174]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

print("\nAccuracy:", test_accuracy)


Accuracy: 0.7777778


# Train model on all data and write predictions to csv

In [27]:
model.fit(X_data, y_labels, epochs=1500, verbose = 2)

Train on 714 samples
Epoch 1/1500
714/714 - 0s - loss: 0.0918 - accuracy: 0.8922
Epoch 2/1500
714/714 - 0s - loss: 0.0863 - accuracy: 0.8936
Epoch 3/1500
714/714 - 0s - loss: 0.0856 - accuracy: 0.8950
Epoch 4/1500
714/714 - 0s - loss: 0.0851 - accuracy: 0.8908
Epoch 5/1500
714/714 - 0s - loss: 0.0834 - accuracy: 0.8978
Epoch 6/1500
714/714 - 0s - loss: 0.0830 - accuracy: 0.8866
Epoch 7/1500
714/714 - 0s - loss: 0.0825 - accuracy: 0.8950
Epoch 8/1500
714/714 - 0s - loss: 0.0815 - accuracy: 0.8866
Epoch 9/1500
714/714 - 0s - loss: 0.0812 - accuracy: 0.8950
Epoch 10/1500
714/714 - 0s - loss: 0.0812 - accuracy: 0.8978
Epoch 11/1500
714/714 - 0s - loss: 0.0794 - accuracy: 0.8978
Epoch 12/1500
714/714 - 0s - loss: 0.0804 - accuracy: 0.8866
Epoch 13/1500
714/714 - 0s - loss: 0.0790 - accuracy: 0.8992
Epoch 14/1500
714/714 - 0s - loss: 0.0801 - accuracy: 0.8964
Epoch 15/1500
714/714 - 0s - loss: 0.0797 - accuracy: 0.8936
Epoch 16/1500
714/714 - 0s - loss: 0.0785 - accuracy: 0.8894
Epoch 17/150

Epoch 132/1500
714/714 - 0s - loss: 0.0711 - accuracy: 0.9020
Epoch 133/1500
714/714 - 0s - loss: 0.0718 - accuracy: 0.9048
Epoch 134/1500
714/714 - 0s - loss: 0.0727 - accuracy: 0.9076
Epoch 135/1500
714/714 - 0s - loss: 0.0709 - accuracy: 0.9076
Epoch 136/1500
714/714 - 0s - loss: 0.0718 - accuracy: 0.9048
Epoch 137/1500
714/714 - 0s - loss: 0.0721 - accuracy: 0.8978
Epoch 138/1500
714/714 - 0s - loss: 0.0726 - accuracy: 0.9076
Epoch 139/1500
714/714 - 0s - loss: 0.0717 - accuracy: 0.9006
Epoch 140/1500
714/714 - 0s - loss: 0.0727 - accuracy: 0.8964
Epoch 141/1500
714/714 - 0s - loss: 0.0722 - accuracy: 0.9020
Epoch 142/1500
714/714 - 0s - loss: 0.0719 - accuracy: 0.9062
Epoch 143/1500
714/714 - 0s - loss: 0.0718 - accuracy: 0.9048
Epoch 144/1500
714/714 - 0s - loss: 0.0717 - accuracy: 0.9146
Epoch 145/1500
714/714 - 0s - loss: 0.0723 - accuracy: 0.9104
Epoch 146/1500
714/714 - 0s - loss: 0.0733 - accuracy: 0.9020
Epoch 147/1500
714/714 - 0s - loss: 0.0706 - accuracy: 0.9062
Epoch 14

714/714 - 0s - loss: 0.0695 - accuracy: 0.9048
Epoch 265/1500
714/714 - 0s - loss: 0.0710 - accuracy: 0.9062
Epoch 266/1500
714/714 - 0s - loss: 0.0701 - accuracy: 0.9090
Epoch 267/1500
714/714 - 0s - loss: 0.0708 - accuracy: 0.9104
Epoch 268/1500
714/714 - 0s - loss: 0.0693 - accuracy: 0.9090
Epoch 269/1500
714/714 - 0s - loss: 0.0707 - accuracy: 0.9146
Epoch 270/1500
714/714 - 0s - loss: 0.0699 - accuracy: 0.9090
Epoch 271/1500
714/714 - 0s - loss: 0.0698 - accuracy: 0.9132
Epoch 272/1500
714/714 - 0s - loss: 0.0705 - accuracy: 0.9118
Epoch 273/1500
714/714 - 0s - loss: 0.0694 - accuracy: 0.9062
Epoch 274/1500
714/714 - 0s - loss: 0.0698 - accuracy: 0.9118
Epoch 275/1500
714/714 - 0s - loss: 0.0704 - accuracy: 0.9062
Epoch 276/1500
714/714 - 0s - loss: 0.0699 - accuracy: 0.9034
Epoch 277/1500
714/714 - 0s - loss: 0.0689 - accuracy: 0.9104
Epoch 278/1500
714/714 - 0s - loss: 0.0700 - accuracy: 0.9020
Epoch 279/1500
714/714 - 0s - loss: 0.0705 - accuracy: 0.9048
Epoch 280/1500
714/714 

Epoch 397/1500
714/714 - 0s - loss: 0.0686 - accuracy: 0.9174
Epoch 398/1500
714/714 - 0s - loss: 0.0689 - accuracy: 0.9118
Epoch 399/1500
714/714 - 0s - loss: 0.0690 - accuracy: 0.9118
Epoch 400/1500
714/714 - 0s - loss: 0.0698 - accuracy: 0.9132
Epoch 401/1500
714/714 - 0s - loss: 0.0677 - accuracy: 0.9132
Epoch 402/1500
714/714 - 0s - loss: 0.0685 - accuracy: 0.9006
Epoch 403/1500
714/714 - 0s - loss: 0.0682 - accuracy: 0.9118
Epoch 404/1500
714/714 - 0s - loss: 0.0684 - accuracy: 0.9118
Epoch 405/1500
714/714 - 0s - loss: 0.0697 - accuracy: 0.9090
Epoch 406/1500
714/714 - 0s - loss: 0.0687 - accuracy: 0.9146
Epoch 407/1500
714/714 - 0s - loss: 0.0675 - accuracy: 0.9132
Epoch 408/1500
714/714 - 0s - loss: 0.0684 - accuracy: 0.9076
Epoch 409/1500
714/714 - 0s - loss: 0.0701 - accuracy: 0.9076
Epoch 410/1500
714/714 - 0s - loss: 0.0691 - accuracy: 0.9146
Epoch 411/1500
714/714 - 0s - loss: 0.0689 - accuracy: 0.9132
Epoch 412/1500
714/714 - 0s - loss: 0.0688 - accuracy: 0.9076
Epoch 41

714/714 - 0s - loss: 0.0666 - accuracy: 0.9062
Epoch 530/1500
714/714 - 0s - loss: 0.0680 - accuracy: 0.9076
Epoch 531/1500
714/714 - 0s - loss: 0.0678 - accuracy: 0.9090
Epoch 532/1500
714/714 - 0s - loss: 0.0673 - accuracy: 0.9076
Epoch 533/1500
714/714 - 0s - loss: 0.0696 - accuracy: 0.9076
Epoch 534/1500
714/714 - 0s - loss: 0.0673 - accuracy: 0.9118
Epoch 535/1500
714/714 - 0s - loss: 0.0683 - accuracy: 0.9174
Epoch 536/1500
714/714 - 0s - loss: 0.0660 - accuracy: 0.9160
Epoch 537/1500
714/714 - 0s - loss: 0.0688 - accuracy: 0.9118
Epoch 538/1500
714/714 - 0s - loss: 0.0679 - accuracy: 0.9104
Epoch 539/1500
714/714 - 0s - loss: 0.0677 - accuracy: 0.9090
Epoch 540/1500
714/714 - 0s - loss: 0.0677 - accuracy: 0.9048
Epoch 541/1500
714/714 - 0s - loss: 0.0686 - accuracy: 0.9104
Epoch 542/1500
714/714 - 0s - loss: 0.0666 - accuracy: 0.9146
Epoch 543/1500
714/714 - 0s - loss: 0.0677 - accuracy: 0.9118
Epoch 544/1500
714/714 - 0s - loss: 0.0685 - accuracy: 0.9132
Epoch 545/1500
714/714 

...

Epoch 927/1500
714/714 - 0s - loss: 0.0650 - accuracy: 0.9146
Epoch 928/1500
714/714 - 0s - loss: 0.0669 - accuracy: 0.9146
Epoch 929/1500
714/714 - 0s - loss: 0.0656 - accuracy: 0.9202
Epoch 930/1500
714/714 - 0s - loss: 0.0661 - accuracy: 0.9160
Epoch 931/1500
714/714 - 0s - loss: 0.0652 - accuracy: 0.9146
Epoch 932/1500
714/714 - 0s - loss: 0.0645 - accuracy: 0.9146
Epoch 933/1500
714/714 - 0s - loss: 0.0661 - accuracy: 0.9174
Epoch 934/1500
714/714 - 0s - loss: 0.0666 - accuracy: 0.9118
Epoch 935/1500
714/714 - 0s - loss: 0.0639 - accuracy: 0.9202
Epoch 936/1500
714/714 - 0s - loss: 0.0636 - accuracy: 0.9202
Epoch 937/1500
714/714 - 0s - loss: 0.0666 - accuracy: 0.9090
Epoch 938/1500
714/714 - 0s - loss: 0.0653 - accuracy: 0.9146
Epoch 939/1500
714/714 - 0s - loss: 0.0656 - accuracy: 0.9146
Epoch 940/1500
714/714 - 0s - loss: 0.0666 - accuracy: 0.9174
Epoch 941/1500
714/714 - 0s - loss: 0.0670 - accuracy: 0.9132
Epoch 942/1500
714/714 - 0s - loss: 0.0645 - accuracy: 0.9174
Epoch 94

714/714 - 0s - loss: 0.0656 - accuracy: 0.9188
Epoch 1059/1500
714/714 - 0s - loss: 0.0660 - accuracy: 0.9118
Epoch 1060/1500
714/714 - 0s - loss: 0.0652 - accuracy: 0.9090
Epoch 1061/1500
714/714 - 0s - loss: 0.0649 - accuracy: 0.9202
Epoch 1062/1500
714/714 - 0s - loss: 0.0646 - accuracy: 0.9174
Epoch 1063/1500
714/714 - 0s - loss: 0.0655 - accuracy: 0.9146
Epoch 1064/1500
714/714 - 0s - loss: 0.0641 - accuracy: 0.9188
Epoch 1065/1500
714/714 - 0s - loss: 0.0651 - accuracy: 0.9146
Epoch 1066/1500
714/714 - 0s - loss: 0.0661 - accuracy: 0.9146
Epoch 1067/1500
714/714 - 0s - loss: 0.0642 - accuracy: 0.9132
Epoch 1068/1500
714/714 - 0s - loss: 0.0647 - accuracy: 0.9146
Epoch 1069/1500
714/714 - 0s - loss: 0.0641 - accuracy: 0.9202
Epoch 1070/1500
714/714 - 0s - loss: 0.0651 - accuracy: 0.9132
Epoch 1071/1500
714/714 - 0s - loss: 0.0658 - accuracy: 0.9132
Epoch 1072/1500
714/714 - 0s - loss: 0.0653 - accuracy: 0.9216
Epoch 1073/1500
714/714 - 0s - loss: 0.0648 - accuracy: 0.9104
Epoch 10

Epoch 1189/1500
714/714 - 0s - loss: 0.0640 - accuracy: 0.9160
Epoch 1190/1500
714/714 - 0s - loss: 0.0636 - accuracy: 0.9160
Epoch 1191/1500
714/714 - 0s - loss: 0.0642 - accuracy: 0.9174
Epoch 1192/1500
714/714 - 0s - loss: 0.0635 - accuracy: 0.9202
Epoch 1193/1500
714/714 - 0s - loss: 0.0643 - accuracy: 0.9118
Epoch 1194/1500
714/714 - 0s - loss: 0.0646 - accuracy: 0.9160
Epoch 1195/1500
714/714 - 0s - loss: 0.0633 - accuracy: 0.9146
Epoch 1196/1500
714/714 - 0s - loss: 0.0643 - accuracy: 0.9160
Epoch 1197/1500
714/714 - 0s - loss: 0.0649 - accuracy: 0.9216
Epoch 1198/1500
714/714 - 0s - loss: 0.0630 - accuracy: 0.9188
Epoch 1199/1500
714/714 - 0s - loss: 0.0664 - accuracy: 0.9160
Epoch 1200/1500
714/714 - 0s - loss: 0.0655 - accuracy: 0.9160
Epoch 1201/1500
714/714 - 0s - loss: 0.0638 - accuracy: 0.9132
Epoch 1202/1500
714/714 - 0s - loss: 0.0643 - accuracy: 0.9118
Epoch 1203/1500
714/714 - 0s - loss: 0.0630 - accuracy: 0.9216
Epoch 1204/1500
714/714 - 0s - loss: 0.0648 - accuracy:

714/714 - 0s - loss: 0.0639 - accuracy: 0.9202
Epoch 1320/1500
714/714 - 0s - loss: 0.0641 - accuracy: 0.9174
Epoch 1321/1500
714/714 - 0s - loss: 0.0638 - accuracy: 0.9146
Epoch 1322/1500
714/714 - 0s - loss: 0.0636 - accuracy: 0.9174
Epoch 1323/1500
714/714 - 0s - loss: 0.0647 - accuracy: 0.9146
Epoch 1324/1500
714/714 - 0s - loss: 0.0638 - accuracy: 0.9188
Epoch 1325/1500
714/714 - 0s - loss: 0.0635 - accuracy: 0.9132
Epoch 1326/1500
714/714 - 0s - loss: 0.0640 - accuracy: 0.9188
Epoch 1327/1500
714/714 - 0s - loss: 0.0634 - accuracy: 0.9174
Epoch 1328/1500
714/714 - 0s - loss: 0.0643 - accuracy: 0.9174
Epoch 1329/1500
714/714 - 0s - loss: 0.0630 - accuracy: 0.9188
Epoch 1330/1500
714/714 - 0s - loss: 0.0630 - accuracy: 0.9174
Epoch 1331/1500
714/714 - 0s - loss: 0.0646 - accuracy: 0.9174
Epoch 1332/1500
714/714 - 0s - loss: 0.0628 - accuracy: 0.9188
Epoch 1333/1500
714/714 - 0s - loss: 0.0657 - accuracy: 0.9230
Epoch 1334/1500
714/714 - 0s - loss: 0.0627 - accuracy: 0.9272
Epoch 13

Epoch 1450/1500
714/714 - 0s - loss: 0.0641 - accuracy: 0.9160
Epoch 1451/1500
714/714 - 0s - loss: 0.0629 - accuracy: 0.9188
Epoch 1452/1500
714/714 - 0s - loss: 0.0620 - accuracy: 0.9258
Epoch 1453/1500
714/714 - 0s - loss: 0.0649 - accuracy: 0.9258
Epoch 1454/1500
714/714 - 0s - loss: 0.0634 - accuracy: 0.9146
Epoch 1455/1500
714/714 - 0s - loss: 0.0634 - accuracy: 0.9202
Epoch 1456/1500
714/714 - 0s - loss: 0.0641 - accuracy: 0.9258
Epoch 1457/1500
714/714 - 0s - loss: 0.0640 - accuracy: 0.9132
Epoch 1458/1500
714/714 - 0s - loss: 0.0627 - accuracy: 0.9174
Epoch 1459/1500
714/714 - 0s - loss: 0.0634 - accuracy: 0.9230
Epoch 1460/1500
714/714 - 0s - loss: 0.0635 - accuracy: 0.9188
Epoch 1461/1500
714/714 - 0s - loss: 0.0634 - accuracy: 0.9188
Epoch 1462/1500
714/714 - 0s - loss: 0.0636 - accuracy: 0.9160
Epoch 1463/1500
714/714 - 0s - loss: 0.0643 - accuracy: 0.9202
Epoch 1464/1500
714/714 - 0s - loss: 0.0639 - accuracy: 0.9132
Epoch 1465/1500
714/714 - 0s - loss: 0.0627 - accuracy:

In [189]:
prob_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

logits = prob_model.predict(X_test)

In [190]:
preds = tf.argmax(input=logits, axis=1)